In [1]:
!pip install 'git+https://github.com/facebookresearch/detectron2.git'
!pip install clearml


  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-0gyd3j8a
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-0gyd3j8a
  Resolved https://github.com/facebookresearch/detectron2.git to commit 0ae803b1449cd2d3f8fa1b7c0f59356db10b3083
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 7.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.2/426.2 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.

In [2]:
# !conda install -c pytorch torchvision cudatoolkit=10.1 pytorch -y 

In [3]:
from kaggle_secrets import UserSecretsClient
 
CLEARML_API_ACCESS_KEY = UserSecretsClient().get_secret('CLEARML_API_ACCESS_KEY')
CLEARML_API_SECRET_KEY = UserSecretsClient().get_secret('CLEARML_API_SECRET_KEY')

import os
os.environ['CLEARML_WEB_HOST'] = 'https://app.clear.ml'
os.environ['CLEARML_API_HOST'] = 'https://api.clear.ml'
os.environ['CLEARML_FILES_HOST'] = 'https://files.clear.ml'
os.environ['CLEARML_API_ACCESS_KEY'] = CLEARML_API_ACCESS_KEY
os.environ['CLEARML_API_SECRET_KEY'] = CLEARML_API_SECRET_KEY

In [4]:
from clearml import Task

# Step 1: Creating a ClearML Task
task = Task.init(
    project_name="drons_vs_berds",
    task_name="detection2_baseline"
)
logger = task.get_logger()

ClearML Task: created new task id=ded117ef272b431ea388f6e315d67f32
2024-05-11 14:42:15,255 - clearml.Repository Detection - WARNING - Failed accessing the jupyter server(s): []
2024-05-11 14:42:15,262 - clearml.Task - INFO - No repository found, storing script code instead


2024-05-11 14:42:17.452789: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-11 14:42:17.452921: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-11 14:42:17.598277: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


ClearML results page: https://app.clear.ml/projects/bda991b03ff34114b2ef93fc1e8d32ba/experiments/ded117ef272b431ea388f6e315d67f32/output/log


In [5]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
# from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [6]:
from detectron2.data.datasets import register_coco_instances
register_coco_instances("my_dataset_train", {}, "/kaggle/input/drons-vs-berds-coco-ann/drons_vs_berds_train.json", "/kaggle/input/images-drons-vs-berds/images")
register_coco_instances("my_dataset_val", {}, "/kaggle/input/drons-vs-berds-coco-ann/drons_vs_berds_valid.json", "/kaggle/input/images-drons-vs-berds/images")
register_coco_instances("my_dataset_test", {}, "/kaggle/input/drons-vs-berds-coco-ann/drons_vs_berds_test.json", "/kaggle/input/images-drons-vs-berds/images")

In [7]:
from detectron2.data import detection_utils as utils
import detectron2.data.transforms as T
import copy
import torch

def custom_mapper(dataset_dict):
    dataset_dict = copy.deepcopy(dataset_dict)  # it will be modified by code below
    image = utils.read_image(dataset_dict["file_name"], format="BGR")
    transform_list = [
        T.Resize((1024,1024)),
        T.RandomBrightness(0.8, 1.8),
        T.RandomContrast(0.6, 1.3),
        T.RandomSaturation(0.8, 1.4),
        T.RandomRotation(angle=[45, 45]),
        T.RandomLighting(0.7),
        T.RandomFlip(prob=0.4, horizontal=False, vertical=True),
    ]
    image, transforms = T.apply_transform_gens(transform_list, image)
    dataset_dict["image"] = torch.as_tensor(image.transpose(2, 0, 1).astype("float32"))

    annos = [
        utils.transform_instance_annotations(obj, transforms, image.shape[:2])
        for obj in dataset_dict.pop("annotations")
        if obj.get("iscrowd", 0) == 0
    ]
    instances = utils.annotations_to_instances(annos, image.shape[:2])
    dataset_dict["instances"] = utils.filter_empty_instances(instances)
    return dataset_dict

from detectron2.engine import DefaultTrainer
from detectron2.data import build_detection_test_loader, build_detection_train_loader
import torch
from detectron2.solver.build import get_default_optimizer_params
from detectron2.solver.build import maybe_add_gradient_clipping

class CustomTrainer(DefaultTrainer):
    @classmethod
    def build_train_loader(cls, cfg):
        return build_detection_train_loader(cfg, mapper=custom_mapper)
    
    # https://medium.com/innovation-res/detectron2-config-optimizer-lr-scheduler-part-1-4555842e1ea
    @classmethod
    def build_optimizer(cls, cfg, model):
        """
        Build an optimizer from config.
        """
        params = get_default_optimizer_params(model)
        return maybe_add_gradient_clipping(cfg, torch.optim.AdamW)(
                          params, 
                          lr=cfg.SOLVER.BASE_LR,        
                          weight_decay=cfg.SOLVER.WEIGHT_DECAY)

In [8]:
COUNT_IMAGE_IN_TRIAN = 40000
EPOCH = 3

In [9]:
import torch
torch.cuda.is_available()


True

In [10]:
from detectron2.engine import DefaultTrainer

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_1x.yaml"))
cfg.DATASETS.TRAIN = ("my_dataset_train",)
cfg.DATASETS.TEST = ("my_dataset_val",)  

cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_1x.yaml")  # Let training initialize from model zoo

cfg.SOLVER.IMS_PER_BATCH = 3  
cfg.SOLVER.MAX_ITER = EPOCH * COUNT_IMAGE_IN_TRIAN // cfg.SOLVER.IMS_PER_BATCH
# cfg.SOLVER.MAX_ITER = 300
cfg.SOLVER.BASE_LR = 3e-4
cfg.SOLVER.LR_SCHEDULER_NAME = "WarmupCosineLR"
cfg.SOLVER.WARMUP_ITERS = int(0.2*cfg.SOLVER.MAX_ITER)

# cfg.SOLVER.IMS_PER_BATCH = 2  # This is the real "batch size" commonly known to deep learning people
# cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
# cfg.SOLVER.MAX_ITER = 200    # 300 iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset
# cfg.SOLVER.STEPS = (100, 200)        # do not decay learning rate
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # The "RoIHead batch size". 128 is faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only has one class (ballon). (see https://detectron2.readthedocs.io/tutorials/datasets.html#update-the-config-for-new-datasets)
# NOTE: this config means the number of classes, but a few popular unofficial tutorials incorrect uses num_classes+1 here.
cfg.INPUT.MIN_SIZE_TRAIN= 576
cfg.INPUT.MAX_SIZE_TRAIN=  1024
cfg.INPUT.MIN_SIZE_TEST=  576
cfg.INPUT.MAX_SIZE_TEST= 1024

cfg.MODEL.ANCHOR_GENERATOR.SIZES = [[16], [32], [64], [128], [256]]
cfg.MODEL.ANCHOR_GENERATOR.ASPECT_RATIOS = [[0.71, 0.5, 0.63]]

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = CustomTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

[05/11 14:42:36 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

model_final_b275ba.pkl: 167MB [00:00, 204MB/s]                           


[05/11 14:42:40 d2.engine.train_loop]: Starting training from iteration 0


/opt/conda/lib/python3.10/site-packages/torch/functional.py:504: UserWarning:

torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /usr/local/src/pytorch/aten/src/ATen/native/TensorShape.cpp:3526.)



[05/11 14:43:01 d2.utils.events]:  eta: 9:00:06  iter: 19  total_loss: 0.8842  loss_cls: 0.8215  loss_box_reg: 0.0003708  loss_rpn_cls: 0.04274  loss_rpn_loc: 0.009695    time: 0.8103  last_time: 0.7996  data_time: 0.0671  last_data_time: 0.0250   lr: 9.4375e-07  max_mem: 7311M
[05/11 14:43:17 d2.utils.events]:  eta: 8:59:50  iter: 39  total_loss: 0.6765  loss_cls: 0.6309  loss_box_reg: 0.0002746  loss_rpn_cls: 0.03327  loss_rpn_loc: 0.007805    time: 0.8106  last_time: 0.8055  data_time: 0.0232  last_data_time: 0.0292   lr: 1.6214e-06  max_mem: 7311M
[05/11 14:43:33 d2.utils.events]:  eta: 8:56:45  iter: 59  total_loss: 0.4643  loss_cls: 0.4244  loss_box_reg: 0.0002754  loss_rpn_cls: 0.03701  loss_rpn_loc: 0.007294    time: 0.8083  last_time: 0.8031  data_time: 0.0224  last_data_time: 0.0229   lr: 2.299e-06  max_mem: 7311M
[05/11 14:43:49 d2.utils.events]:  eta: 8:56:25  iter: 79  total_loss: 0.3165  loss_cls: 0.2363  loss_box_reg: 0.0003571  loss_rpn_cls: 0.03232  loss_rpn_loc: 0.008

In [11]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set a custom testing threshold
predictor = DefaultPredictor(cfg)

[05/11 23:41:05 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from ./output/model_final.pth ...
2024-05-11 23:41:06,437 - clearml.model - INFO - Selected model id: 007b47a500b74102b6935e3af868213a


In [12]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
evaluator = COCOEvaluator("my_dataset_test", output_dir="./output_test")
val_loader = build_detection_test_loader(cfg, "my_dataset_test")
 
result = inference_on_dataset(predictor.model, val_loader, evaluator)

for name, value in result['bbox'].items():
    logger.report_single_value(name=f"test_{name}",value=value)

[05/11 23:41:08 d2.data.datasets.coco]: Loaded 3116 images in COCO format from /kaggle/input/drons-vs-berds-coco-ann/drons_vs_berds_test.json
[05/11 23:41:08 d2.data.build]: Distribution of instances among all 1 categories:
|  category  | #instances   |
|:----------:|:-------------|
|   drone    | 3729         |
|            |              |
[05/11 23:41:08 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(576, 576), max_size=1024, sample_style='choice')]
[05/11 23:41:08 d2.data.common]: Serializing the dataset using: <class 'detectron2.data.common._TorchSerializedList'>
[05/11 23:41:08 d2.data.common]: Serializing 3116 elements to byte tensors and concatenating them all ...
[05/11 23:41:08 d2.data.common]: Serialized dataset takes 0.81 MiB
[05/11 23:41:08 d2.evaluation.evaluator]: Start inference on 3116 batches
[05/11 23:41:09 d2.evaluation.evaluator]: Inference done 11/3116. Dataloading: 0.0070 s/iter. Inference: 0.0479 

In [13]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
evaluator = COCOEvaluator("my_dataset_val", output_dir="./output_val")
val_loader = build_detection_test_loader(cfg, "my_dataset_val")
 
result = inference_on_dataset(predictor.model, val_loader, evaluator)

for name, value in result['bbox'].items():
    logger.report_single_value(name=f"val_{name}",value=value)

[05/11 23:44:03 d2.data.datasets.coco]: Loaded 7805 images in COCO format from /kaggle/input/drons-vs-berds-coco-ann/drons_vs_berds_valid.json
[05/11 23:44:04 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(576, 576), max_size=1024, sample_style='choice')]
[05/11 23:44:04 d2.data.common]: Serializing the dataset using: <class 'detectron2.data.common._TorchSerializedList'>
[05/11 23:44:04 d2.data.common]: Serializing 7805 elements to byte tensors and concatenating them all ...
[05/11 23:44:04 d2.data.common]: Serialized dataset takes 1.93 MiB
[05/11 23:44:04 d2.evaluation.evaluator]: Start inference on 7805 batches
[05/11 23:44:05 d2.evaluation.evaluator]: Inference done 11/7805. Dataloading: 0.0179 s/iter. Inference: 0.0495 s/iter. Eval: 0.0003 s/iter. Total: 0.0677 s/iter. ETA=0:08:47
[05/11 23:44:10 d2.evaluation.evaluator]: Inference done 98/7805. Dataloading: 0.0100 s/iter. Inference: 0.0485 s/iter. Eval: 0.0003 s/ite